<a href="https://colab.research.google.com/github/akiitr/python/blob/master/ml_hackathon_qc_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import numpy as np
import csv

In [113]:
# Loading the database
scale = StandardScaler()
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25920 entries, 0 to 25919
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID1                        25920 non-null  int64  
 1   Specific ailments          25920 non-null  int64  
 2   ID2                        25920 non-null  int64  
 3   Food preference            25910 non-null  object 
 4   Age                        25920 non-null  int64  
 5   BMI                        25920 non-null  float64
 6   Smoker?                    25920 non-null  object 
 7   Living in?                 25920 non-null  object 
 8   Any heriditary condition?  25920 non-null  object 
 9   Follow Diet                24947 non-null  float64
 10  Physical activity          24947 non-null  float64
 11  Regular sleeping hours     24947 non-null  float64
 12  Alcohol consumption        24947 non-null  float64
 13  Social interaction         24947 non-null  flo

,ID1,Specific ailments,ID2,Food preference,Age,BMI,Smoker?,Living in?,Any heriditary condition?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,Taking supplements,Mental health management,Illness count last year,Healthy
0,9563,3,1,DX4,30,25.890033,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,26429,33,1,DX4,30,25.890033,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,27772,34,2,DX6,39,29.921864,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,8301,29,3,DX1,19,25.242091,YES,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,11732,37,3,DX4,19,25.242091,YES,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [114]:
# Train Data base cleaning
#df.info()
#Replacing the col names without space and ?
df.rename(columns = {'Specific ailments':'Ailments','Food preference':'Food','Smoker?':'Smoker','Living in?':'Living','Any heriditary condition?':'Heriditary','Follow Diet':'Diet','Physical activity':'Physical','Regular sleeping hours':'Sleep','Alcohol consumption':'Alcohol','Social interaction':'Social','Taking supplements':'Supplements','Mental health management':'Mental','Illness count last year':'Illness_no_last_year'},inplace=True)
#Replacing all the empty values to NAN
df.replace('',np.nan, inplace=True)
#Dropping all the empty values to create meaningfull data
df.dropna(inplace=True)
#Dropping ID1 and ID2 as these are just row identifiers
df.drop(['ID1','ID2'],axis=1,inplace=True)
#Dropping Food Preference as it is not making any sense
#df.drop('Food',axis=1,inplace=True)
df['Food'] = df.Food.map({'DX1 ' : 1, 'DX1 DX2 ' : 2, 'DX1 DX2 DX3 ' : 3, 'DX1 DX2 DX3 DX4 ' : 4, 'DX1 DX2 DX3 DX4 DX5 ' : 5, 'DX1 DX2 DX3 DX5 ' : 6, 'DX1 DX2 DX4 ' : 7, 'DX1 DX2 DX4 DX5 ' : 8, 'DX1 DX2 DX5 ' : 9, 'DX1 DX3 ' : 10, 'DX1 DX3 DX4 ' : 11, 'DX1 DX3 DX4 DX5 ' : 12, 'DX1 DX3 DX5 ' : 13, 'DX1 DX4 ' : 14, 'DX1 DX4 DX5 ' : 15, 'DX1 DX5 ' : 16, 'DX2 ' : 17, 'DX2 DX3 ' : 18, 'DX2 DX3 DX4 ' : 19, 'DX2 DX3 DX4 DX5 ' : 20, 'DX2 DX3 DX5 ' : 21, 'DX2 DX4 ' : 22, 'DX2 DX4 DX5 ' : 23, 'DX2 DX5 ' : 24, 'DX3 ' : 25, 'DX3 DX4 ' : 26, 'DX3 DX4 DX5 ' : 27, 'DX3 DX5 ' : 28, 'DX4 ' : 29, 'DX4 DX5 ' : 30, 'DX5 ' : 31, 'DX6' : 32})
#Changing to an integar value
df['Smoker'] = df.Smoker.map({'NO': 0, 'YES': 1, 'Cannot say': 1})
df['Living'] = df.Living.map({'RURAL': 0, 'URBAN': 1})
#Dropping Heriditary condition as all row have only one option stable
df.drop('Heriditary',axis=1,inplace=True)


df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24937 entries, 0 to 25909
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ailments              24937 non-null  int64  
 1   Food                  24937 non-null  int64  
 2   Age                   24937 non-null  int64  
 3   BMI                   24937 non-null  float64
 4   Smoker                24937 non-null  int64  
 5   Living                24937 non-null  int64  
 6   Diet                  24937 non-null  float64
 7   Physical              24937 non-null  float64
 8   Sleep                 24937 non-null  float64
 9   Alcohol               24937 non-null  float64
 10  Social                24937 non-null  float64
 11  Supplements           24937 non-null  float64
 12  Mental                24937 non-null  float64
 13  Illness_no_last_year  24937 non-null  float64
 14  Healthy               24937 non-null  int64  
dtypes: float64(9), int6

,Ailments,Food,Age,BMI,Smoker,Living,Diet,Physical,Sleep,Alcohol,Social,Supplements,Mental,Illness_no_last_year,Healthy
0,3,29,30,25.890033,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,33,29,30,25.890033,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,34,32,39,29.921864,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,29,1,19,25.242091,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,37,29,19,25.242091,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [115]:
# Test Data base cleaning

#Replacing the col names without space and ?
df_test.rename(columns = {'Specific ailments':'Ailments','Food preference':'Food','Smoker?':'Smoker','Living in?':'Living','Any heriditary condition?':'Heriditary','Follow Diet':'Diet','Physical activity':'Physical','Regular sleeping hours':'Sleep','Alcohol consumption':'Alcohol','Social interaction':'Social','Taking supplements':'Supplements','Mental health management':'Mental','Illness count last year':'Illness_no_last_year'},inplace=True)
#Replacing all the empty values to NAN
df_test.replace('',np.nan, inplace=True)
#Dropping all the empty values to create meaningfull data
#df_test.dropna(inplace=True)
#Dropping ID1 and ID2 as these are just row identifiers
df_test.drop(['ID1','ID2'],axis=1,inplace=True)
#Dropping Food Preference as it is not making any sense
#df.drop('Food',axis=1,inplace=True)
df_test['Food'] = df_test.Food.map({'DX1 ' : 1, 'DX1 DX2 ' : 2, 'DX1 DX2 DX3 ' : 3, 'DX1 DX2 DX3 DX4 ' : 4, 'DX1 DX2 DX3 DX4 DX5 ' : 5, 'DX1 DX2 DX3 DX5 ' : 6, 'DX1 DX2 DX4 ' : 7, 'DX1 DX2 DX4 DX5 ' : 8, 'DX1 DX2 DX5 ' : 9, 'DX1 DX3 ' : 10, 'DX1 DX3 DX4 ' : 11, 'DX1 DX3 DX4 DX5 ' : 12, 'DX1 DX3 DX5 ' : 13, 'DX1 DX4 ' : 14, 'DX1 DX4 DX5 ' : 15, 'DX1 DX5 ' : 16, 'DX2 ' : 17, 'DX2 DX3 ' : 18, 'DX2 DX3 DX4 ' : 19, 'DX2 DX3 DX4 DX5 ' : 20, 'DX2 DX3 DX5 ' : 21, 'DX2 DX4 ' : 22, 'DX2 DX4 DX5 ' : 23, 'DX2 DX5 ' : 24, 'DX3 ' : 25, 'DX3 DX4 ' : 26, 'DX3 DX4 DX5 ' : 27, 'DX3 DX5 ' : 28, 'DX4 ' : 29, 'DX4 DX5 ' : 30, 'DX5 ' : 31, 'DX6' : 32})
#Changing to an integar value
df_test['Smoker'] = df_test.Smoker.map({'NO': 0, 'YES': 1, 'Cannot say': 1})
df_test['Living'] = df_test.Living.map({'RURAL': 0, 'URBAN': 1})
#Dropping Heriditary condition as all row have only one option stable
df_test.drop('Heriditary',axis=1,inplace=True)



df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6480 entries, 0 to 6479
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Ailments              6480 non-null   int64  
 1   Food                  6477 non-null   float64
 2   Age                   6480 non-null   int64  
 3   BMI                   6480 non-null   float64
 4   Smoker                6480 non-null   int64  
 5   Living                6480 non-null   int64  
 6   Diet                  6218 non-null   float64
 7   Physical              6218 non-null   float64
 8   Sleep                 6218 non-null   float64
 9   Alcohol               6218 non-null   float64
 10  Social                6218 non-null   float64
 11  Supplements           6218 non-null   float64
 12  Mental                6218 non-null   float64
 13  Illness_no_last_year  6218 non-null   float64
dtypes: float64(10), int64(4)
memory usage: 708.9 KB


,Ailments,Food,Age,BMI,Smoker,Living,Diet,Physical,Sleep,Alcohol,Social,Supplements,Mental,Illness_no_last_year
0,2,32.0,38,18.879331,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,5,1.0,46,21.231991,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,37,29.0,11,17.867876,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,44,31.0,10,26.886096,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
4,37,26.0,3,23.362746,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0


In [116]:
#Dividing the data for test and train

#Creating Input and output data
x = df[list(df.columns[0:13])]
y = df[['Healthy']]

#Splitting the data for train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [117]:
# train dataset
train = xgb.DMatrix(x_train,label = y_train)
# test dataset
test = xgb.DMatrix (x_test, label = y_test)

In [118]:
#XGB Model

# Parameters for the training
param = {
    'max_depth':8,
    'learning_rate': 0.45,
    'objective': 'multi:softmax',
    'num_class':2,
}
epochs = 10

# Training the model
model = xgb.train(param, train, epochs)
model
# checking the prediction and accuracy
predictions = model.predict(test).astype(int)
print(predictions)
print(accuracy_score(y_test, predictions))

[1 1 1 ... 1 1 1]
0.8217722534081796


In [119]:
# Submission data preparation for output CSV
xt = df_test[list(df_test.columns[0:13])]
xt_test = scale.fit_transform(xt)
test2 = xgb.DMatrix (xt_test)
predictions = model.predict(test2).astype(int)
ak = pd.DataFrame(predictions, columns=['predictions'])
ak.to_csv('predictions_beginner.csv', index=None)
print(predictions)
ak.info()
ak.head()

[1 1 1 ... 1 0 0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6480 entries, 0 to 6479
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   predictions  6480 non-null   int64
dtypes: int64(1)
memory usage: 50.8 KB


,predictions
0,1
1,1
2,1
3,1
4,1


In [120]:
# Low accuracy trying randomforest classifier model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier(n_estimators = 1000)
model = clf.fit(x_train,y_train.values.ravel())
# checking the prediction and accuracy
output = clf.predict(x_test)
print(output)
print(accuracy_score(y_test, output))

[1 1 1 ... 1 1 1]
0.8061347233360064


In [121]:
# Trying Gaussian Classifier
clf = GaussianNB()
model = clf.fit(x_train,y_train.values.ravel())

prediction = clf.predict(x_test)
print(accuracy_score(y_test, prediction))

0.6978748997594226


In [122]:
# Trying Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train, y_train.values.ravel())
predict = classifier.predict(x_test)
print(accuracy_score(y_test, predict))

0.6858460304731355
